#WeatherPy

In [2]:
# Setup and Importing Dependencies
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import requests
import scipy.stats as st
from scipy.stats import linregress
import time

In [3]:
# Imports and defines API Key
from api_keys import weather_api_key
api_key = weather_api_key

##Generate Cities List

In [4]:
# Creating list to be filled with cities
from citipy import citipy
citylist = []

# Gathers random list of coordinates(Lat,Lng) and adds it to list
randomlatlist= np.random.uniform(-90,90,1600)
randomlonglist= np.random.uniform(-180,180,1600)
coordlist = [(randomlatlist[i], randomlonglist[i]) for i in range(0, len(randomlatlist))]

# Takes a look at the latitude and longitude and finds the nearest city
for coord in coordlist:
    city = citipy.nearest_city(coord[0], coord[1]).city_name
    
    #If this city is a unique city not yet in the citylist, this adds it to the city list
    if city not in citylist:
        citylist.append(city)

# Prints number of cities. This is to make sure I've collected at least 500 cities to create data from.
print(len(citylist))

642


###Perform API Calls
* Performs a weather check on each city using a series of successive API calls.
* Includes a print log of each city as it's being processed (with both the city number and city name).

In [5]:
# Creating list to filled with the cities' data
cities_data = []

# Creates base of url
base = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + api_key

# Begins both the record count and the set count at 1
rec_count = 1
set_count = 1

# Iterates through each city in the city list at a set limit and time delay
for i, city in enumerate(citylist):
    time.sleep(0.25)
    # After 50 cities, resets the record count to 0 and restarts the process.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        rec_count = 0
        
    # Prints City's Name along with its Record and Set Count
    print(f"Processing Record {rec_count} of set {set_count} | {city}")
    
    # Adds 1 to Record Count
    rec_count += 1
    url = f"{base}&q={city}"
    
    # Performs requests for API data to retrieve all variables needed
    try:
        weather_data = requests.get(url).json()
        city_tmp = weather_data['main']['temp']
        city_max_tmp = weather_data['main']['temp_max']
        city_humid = weather_data['main']['humidity']
        longitude = weather_data['coord']['lon']
        latitude = weather_data['coord']['lat']
        city_clouds = weather_data['clouds']['all']
        country = weather_data['sys']['country']
        city_wind_spd = weather_data['wind']['speed']
        date = weather_data['dt']
        
        # Add data requested into cities_data and convert to dictionary
        cities_data.append({"City" : city,
                           "Temperature" : city_max_tmp,
                           "Humidity" : city_humid,
                           "Longitude" : longitude,
                           "Latitude" : latitude,
                           "Cloudiness" : city_clouds,
                           "Country" :  country,
                           "Windspeed": city_wind_spd,
                           "Date": date
                           })
    # Adds exceptions when city is not found to keep the code running
    except:
        # Adds print statement for when exception is found
        print(f'City not found. Skipping...')
        pass

# Prints when its done running
print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")


Processing Record 40 of set 4 | nizhniy odes
Processing Record 41 of set 4 | port blair
Processing Record 42 of set 4 | ballina
Processing Record 43 of set 4 | abu samrah
Processing Record 44 of set 4 | dharan
Processing Record 45 of set 4 | berbera
Processing Record 46 of set 4 | suhbaatar
Processing Record 47 of set 4 | monrovia
Processing Record 48 of set 4 | churapcha
Processing Record 49 of set 4 | bellavista
Processing Record 0 of set 5 | vlore
Processing Record 1 of set 5 | coracora
Processing Record 2 of set 5 | amderma
City not found. Skipping...
Processing Record 3 of set 5 | saint-augustin
Processing Record 4 of set 5 | porto novo
Processing Record 5 of set 5 | coquimbo
Processing Record 6 of set 5 | itarema
Processing Record 7 of set 5 | arlit
Processing Record 8 of set 5 | iqaluit
Processing Record 9 of set 5 | hofn
Processing Record 10 of set 5 | gat
Processing Record 11 of set 5 | sanda
Processing Record 12 of set 5 | rosa zarate
Processing Record 13 of set 5 | dusti
Pr

###Convert Raw Data to DataFrame
* Exports the above city data into a .csv
* Displays the DataFrame

In [6]:
# Create Dataframe from API Data Collected
weatherpy_df = pd.DataFrame.from_dict(cities_data)

# Saves above data to CSV
weatherpy_csv = weatherpy_df.to_csv("Output_Data/City_Weather.csv",
                                   encoding="utf-8", index=True)

# Prints Head of DataFrame
weatherpy_df.head()

,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,rikitea,74.71,80,-134.97,-23.12,100,PF,23.55,1603666646
1,axim,77.76,90,-2.24,4.87,100,GH,7.27,1603666654
2,ponta do sol,66.20,68,-17.10,32.67,20,PT,6.93,1603666571
3,busselton,64.00,58,115.33,-33.65,0,AU,5.48,1603666694
4,albany,44.01,55,-73.97,42.60,0,US,4.45,1603666520


##Data Cleanup
* Inspects the data and removes any cities where the humidity > 100%
* Note, if there were no cities above that have humidity > 100%, this process is then skipped.

##Plotting the Data
Creates scatter plots that showcases the following relationships:
* Latitude vs. Temperature (F)
* Latitude vs. Humidity (%)
* Latitude vs. Cloudiness (%)
* Wind Speed (mph) vs. Latitude

###Latitude vs. Temperature Plot

###Latitude vs. Humidity Plot

###Latitude vs. Cloudiness Plot

###Latitude vs. Wind Speed Plot

##Linear Regression

###North Hemisphere - Max Temp (F) vs. Latitude Linear Regression

###South Hemisphere - Max Temp (F) vs. Latitude Linear Regression

###North Hemisphere - Humidity (%) vs. Latitude Linear Regression

###South Hemisphere - Humidity (%) vs. Latitude Linear Regression

###North Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

###South Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

###North Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

###South Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression